# Customizable object detector model with TF1 API

## Workspace preparation

In [2]:
%tensorflow_version 1.x

! git clone --depth 1 https://github.com/tensorflow/models
! pip install -q tf_slim
! pip install -qq pycocotools

%cd /content/models/research/
! protoc object_detection/protos/*.proto --python_out=.

from google.colab import drive
drive.mount('/content/drive')
!rm -rf /content/sample_data

TensorFlow 1.x selected.
Cloning into 'models'...
remote: Enumerating objects: 2232, done.
remote: Counting objects: 100% (2232/2232), done.
remote: Compressing objects: 100% (1930/1930), done.
remote: Total 2232 (delta 541), reused 976 (delta 277), pack-reused 0
Receiving objects: 100% (2232/2232), 30.48 MiB | 24.54 MiB/s, done.
Resolving deltas: 100% (541/541), done.
     |████████████████████████████████| 358kB 5.1MB/s 
/content/models/research
Mounted at /content/drive


In [3]:
import os
import time

os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/:/content/models/research/slim/'

from google.protobuf import text_format
from object_detection.utils import config_util

%cd /content/drive/My Drive/projects/ML/object_detection_tf/workspace

/content/drive/My Drive/projects/ML/object_detection_tf/workspace


## Model selection

In [4]:
selected_model = "faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28" #@param ["ssd_mobilenet_v2_quantized_300x300", "ssd_mobilenet_v3_small_coco", "ssdlite_mobilenet_edgetpu_coco_quant", "faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"]
pipeline_path = f'my_models/pipelines/{selected_model}.config'
output_model_dir = f'my_models/{selected_model}/'

## Pipeline adjustments

In [5]:
# Getting configs from pipeline file
configs = config_util.get_configs_from_pipeline_file(pipeline_path)
pipeline_config = config_util.create_pipeline_proto_from_configs(configs)
pipeline_config

model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: "faster_rcnn_inception_resnet_v2"
      first_stage_features_stride: 8
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        height_stride: 8
        width_stride: 8
        scales: 0.25
        scales: 0.5
        scales: 1.0
        scales: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 1.0
        aspect_ratios: 2.0
      }
    }
    first_stage_atrous_rate: 2
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.009990000165998936
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.6990000009536743
    first_stage_max_proposals: 100

In [6]:
%%writefile {pipeline_path}

model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer { 
        min_dimension: 600 
        max_dimension: 1024 
      }
    }
    feature_extractor {
      type: "faster_rcnn_inception_resnet_v2"
      first_stage_features_stride: 8
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        height_stride: 8
        width_stride: 8
        scales: 0.25
        scales: 0.5
        scales: 1.0
        scales: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 1.0
        aspect_ratios: 2.0
      }
    }
    first_stage_atrous_rate: 2
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.00999
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.699
    first_stage_max_proposals: 1000
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 17
    maxpool_kernel_size: 1
    maxpool_stride: 1
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
        use_dropout: false
        dropout_keep_probability: 1.0
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.3
        iou_threshold: 0.6
        max_detections_per_class: 800
        max_total_detections: 800
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}
train_config {
  batch_size: 1
  max_number_of_boxes: 800
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  optimizer {
    momentum_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.04
          total_steps: 10000
          warmup_learning_rate: 0.013
          warmup_steps: 100
        }
      }
      momentum_optimizer_value: 0.8999
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "pre_trained_models/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/ckpt-0"
  from_detection_checkpoint: true
  num_steps: 10000
}
train_input_reader {
  label_map_path: "annotations/label_map.pbtxt"
  tf_record_input_reader {
    input_path: "annotations/train.tfrecord"
  }
  max_number_of_boxes: 800

}
eval_config {
  batch_size: 1
  use_moving_averages: false
  max_num_boxes_to_visualize: 800
}
eval_input_reader {
  label_map_path: "annotations/label_map.pbtxt"
  shuffle: false
  num_readers: 1
  tf_record_input_reader {
    input_path: "annotations/test.tfrecord"
  }
  max_number_of_boxes: 800

}


Overwriting my_models/pipelines/faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28.config


## Training the model

### Training the model


In [ ]:
start_time = time.time()

!python3 /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_path}\
    --model_dir={output_model_dir} \
    --num_train_steps=10000 \
    --sample_1_of_n_eval_examples=1 \
    --alsologtostderr

end_time = time.time()
elapsed_time = end_time - start_time
print(f'Elapsed time: {int(elapsed_time/60)}m')

W1008 22:44:37.358770 140205958432640 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 10000
I1008 22:44:37.358992 140205958432640 config_util.py:552] Maybe overwriting train_steps: 10000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1008 22:44:37.359091 140205958432640 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I1008 22:44:37.359170 140205958432640 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1008 22:44:37.359249 140205958432640 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1008 22:44:37.359371 140205958432640 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to